# Python for AI Projects

## Introduction

**Generative AI and Agentic AI**

In this Jupyter notebook - we'll quickly setup our Python environment and see how we can run a series of Streamlit dashboards within our Google Colab workspace.

![example-python-rag-app](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/app-python-rag.png)

### Basic Application Flow

We will be implementing 3 different versions of our Streamlit application using the following design with pure Python, `LangChain` and `LangGraph`

![design-basic-app](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/design-basic-app.png)

### Advanced Application Flow

Finally we'll extend our application with more advanced components building on top of the `LangGraph` example to create something which more closely mimics what we might expect from a chat app like ChatGPT!

![design-advanced-app](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/design-advanced-app.png)

### Source Code

* [Entire GitHub Repo](https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/tree/main)
* [Hello World Streamlit App](https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/blob/main/streamlit-hello-world.py)
* [Pure Python AI RAG App](https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/blob/main/streamlit-python-rag.py)
* [LangChain AI RAG App](https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/blob/main/streamlit-langchain-rag.py)
* [LangGraph AI RAG App](https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/blob/main/streamlit-langgraph-rag.py)
* [Advanced LangGraph AI RAG + ML Models App](https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/blob/main/streamlit-advanced-ai-app.py)

**⚠️ BEFORE YOU BEGIN!**

Make sure you’re using the **GPU runtime** in Google Colab for better performance when running local language models like TinyLlama.

To enable GPU runtime, please go to the menu:  
**Runtime → Change runtime type → Hardware accelerator → GPU**

### Executing Code Cells

To execute each cell in this notebook - you can click on the play button on the left of each cell or hit `command/shift + enter` when you're navigating to the cell.

### Setup and Installation

In the following cell we'll run a few commands to install the required Python packages and also grab all of our Streamlit application code and data artefacts from our GitHub repository.

The cell below should around ~2 minutes to run as we'll need to download and cache a few large-language models from Hugging Face!

In [ ]:

# ====================
# Initial setup steps
# ====================

# Install Python libraries
!pip install --quiet faiss-cpu==1.11.0
!pip install --quiet ctransformers==0.2.27
!pip install --quiet dotenv==0.9.9
!pip install --quiet pyngrok==7.2.12
!pip install --quiet streamlit==1.47.1
!pip install --quiet langchain_openai==0.3.28
!pip install --quiet openai==1.98.0
!pip install --quiet langchain==0.3.27
!pip install --quiet langchain-community==0.3.27
!pip install --quiet langchain-huggingface==0.3.1
!pip install --quiet langgraph==0.6.2

# Clone GitHub repo into a "data" folder
!git clone https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259.git data

# Need to change directory into "data" to download git lfs data objects
%cd data
!git lfs pull

# Then we need to change directory back up so all our paths are correct
%cd ..

# Turn off future warnings for cleaner output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Frontload the necessary LLM and AI libraries we'll be using in this tutorial
from sentence_transformers import SentenceTransformer
from huggingface_hub import hf_hub_download, snapshot_download
import os

cache_root = "data/models"

# this cache_dir is important as we'll reuse this in some of our Streamlit apps
tiny_llama_model_path = hf_hub_download(
    repo_id="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
    filename="tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
    cache_dir=cache_root
)

# Download and cache our embedding model
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
cached_embedding_model_path = snapshot_download(
  repo_id=embedding_model_id,
  cache_dir=cache_root
)

# We'll use this path to refer to our cached embedding model in our Streamlit apps
print(cached_embedding_model_path)
print("✅ Initial setup steps complete!")

# 1. Introduction to Streamlit

Welcome to this hands-on tutorial! In this notebook, we’ll be using Streamlit, a lightweight Python framework for building interactive web dashboards and AI apps — all with minimal code.

## 1.1 What is Streamlit?

Streamlit lets you turn any Python script into a shareable web app using simple commands like st.write(), st.text_input(), and st.button().

It's perfect for:

- Visualizing data and model predictions
- Prototyping dashboards
- Building AI-powered tools quickly

## 1.2 The Challenge in Google Colab

Because Colab runs on remote virtual machines, we can’t directly access localhost ports like we would on our own computers.

That means even if we run:

```python
streamlit run app.py
```

We still won't be able to visit localhost:8501 in our browser to view our application!

## 1.3 Ngrok for Secure Tunnels

To solve this, we’ll use Ngrok, a tool that creates a secure tunnel from the internet to your Colab machine.

We’ll use the following pattern for all of our Streamlit apps in this tutorial:

- Start running our Streamlit apps on port `8051` (we’ll avoid `8501` to reduce potential conflicts)
- Use Ngrok to expose port 8051 to a public web URL
- Visit the generated public URL to view our live dashboards

# 2. Pre-flight Checks

We will need to setup a few secure access components before we can proceed with our Streamlit and AI agents tutorial.

## 2.1 Setup Ngrok Account

1. Create Ngrok account at https://dashboard.ngrok.com/signup
2. Acquire your `AuthToken` at https://dashboard.ngrok.com/get-started/your-authtoken

![ngrok-copy-auth-token](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/ngrok-copy-auth-token.png)

## 2.2 Setup OpenRouter Account

1. Create OpenRouter account at https://openrouter.ai/sign-up
2. Create an API token at https://openrouter.ai/settings/keys with **read only** access and spend limit set to $0

![open-router-create-api-key](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/open-router-create-api-key.png)

3. Store this API key safely as you won't be able to see it again once you click away

> **WARNING 🚨🚨🚨** don't be like me and share your key publicly like in this image! This example is just for illustration and I've already deleted this set of API keys on my account!!!!!!! 🥵

![open-router-copy-api-key](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/open-router-copy-api-key.png)

## 2.3 Streamlit Password

We'll also secure our Streamlit apps by using a simple authentication password. This ensures that our publicly available URL still has a layer of authentication in-memory so only we can access it.

Simply decide on a simple password for your Streamlit applications - but note that you'll need to type this when we start viewing our dashboards, so I wouldn't make it too long or complicated for a better learning experience!

## 2.4 Setup Authentication

Run the code cell immediately below to populate a local `.env` file with your sensitive information we setup in the previous step!

Make sure you have the following ready so you can paste it in when prompted in the following cell:

* Ngrok Auth Token
* OpenRouter API Key
* Streamlit password (remember to keep it short as we'll need to type it for every Streamlit app we run!)

We’ll ask for these values securely using getpass, and store them in a local `.env` file so they’re not exposed in the notebook.

> **Warning 🚨🚨🚨** Make sure to never share your Auth and API keys and be careful to avoid exposing them to the public especially when commiting files into GitHub! I've also setup a secure cleaup step to make sure we remove the `.env` file at [bottom of this notebook!](#final-steps)

In [ ]:
# =========================
# Secure .env Setup Script
# =========================
import getpass
import os
import dotenv

# Step 1: Check if .env file exists
if os.path.exists(".env"):
    print("✅ .env file already exists!")

    # Load existing environment variables
    dotenv.load_dotenv()

    # Check each required variable individually
    ngrok_token = os.getenv("NGROK_AUTH_TOKEN")
    if not ngrok_token:
        ngrok_token = getpass.getpass("🔐 Paste your NGROK_AUTH_TOKEN: ")
        with open(".env", "a") as f:
            f.write(f'NGROK_AUTH_TOKEN={ngrok_token}\n')

    openrouter_key = os.getenv("OPEN_ROUTER_API_KEY")
    if not openrouter_key:
        openrouter_key = getpass.getpass("🔐 Paste your OPEN_ROUTER_API_KEY: ")
        with open(".env", "a") as f:
            f.write(f'OPEN_ROUTER_API_KEY={openrouter_key}\n')

    streamlit_password = os.getenv("STREAMLIT_PASSWORD")
    if not streamlit_password:
        password_input = getpass.getpass("🔐 Set your STREAMLIT_PASSWORD (press Enter to use default): ")
        streamlit_password = password_input or "linkedin-learning"
        if password_input == "":
            print(f"ℹ️ No password entered — using default: {streamlit_password}")
        else:
            print("✅ Streamlit password set!")
        with open(".env", "a") as f:
            f.write(f'STREAMLIT_PASSWORD={streamlit_password}\n')

    print("✅ All required environment variables are now set!")

# Step 2: If .env does not exist, prompt for everything
else:
    print("⚙️ .env file not found — let's create one now!")

    ngrok_token = getpass.getpass("🔐 Paste your NGROK_AUTH_TOKEN: ")
    openrouter_key = getpass.getpass("🔐 Paste your OPEN_ROUTER_API_KEY: ")
    password_input = getpass.getpass("🔐 Set your STREAMLIT_PASSWORD (press Enter to use default): ")

    streamlit_password = password_input or "linkedin-learning"
    if password_input == "":
        print(f"ℹ️ No password entered — using default: {streamlit_password}")
    else:
        print("✅ Streamlit password set! (hidden)")

    with open(".env", "w") as f:
        f.write(f'NGROK_AUTH_TOKEN={ngrok_token}\n')
        f.write(f'OPEN_ROUTER_API_KEY={openrouter_key}\n')
        f.write(f'STREAMLIT_PASSWORD={streamlit_password}\n')

    print("✅ .env file created securely!")

# Step 3: Append model paths to .env
with open(".env", "a") as f:
    f.write(f'TINY_LLAMA_MODEL_PATH={tiny_llama_model_path}\n')
    f.write(f'EMBEDDING_MODEL_PATH={cached_embedding_model_path}\n')
print("✅ Local model paths appended to .env file!")

# Finally - load in the newly created environment variables
dotenv.load_dotenv()
print("✅ Latest .env file successfully loaded!")

# 3. Streamlit Hello World App

In the next section, we’ll demonstrate how to run a basic Streamlit app to help verify everything is working smoothly before we dive deeper into our GenAI Streamlit applications.

## 3.1 Streamlit Helper Functions

We'll firstly define some Python helper functions to help us launch and shutdown our Streamlit applications using Ngrok to host our live dashboards.

You can run the cell directly below to initialize these functions:

* `launch_streamlit_in_colab`
* `shutdown_streamlit_and_ngrok`

In [ ]:
def launch_streamlit_in_colab(app_path: str):
    """
    Launch a Streamlit app within Google Colab using an ngrok tunnel.

    Parameters:
    ----------
    app_path : str
        Path to the Streamlit `.py` app file to be launched (e.g., "app.py")

    Returns:
    -------
    public_url : str
        A publicly accessible ngrok URL that points to the Streamlit app running on port 8501.

    This function performs:
    ------------------------
    - Authenticates with ngrok using a token stored in the .env file (`NGROK_AUTH_TOKEN`)
    - Kills any existing ngrok tunnels
    - Launches a new tunnel to port 8501
    - Starts the Streamlit app in the background
    - Waits briefly to allow the app to start up
    """

    import os
    import time
    from pyngrok import ngrok

    # ----------------------------
    # Set ngrok authentication token
    # ----------------------------
    ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))

    # ----------------------------
    # Kill any existing tunnels
    # ----------------------------
    ngrok.kill()

    # ----------------------------
    # Open a new tunnel to port 8501
    # ----------------------------
    public_url = ngrok.connect(8501, "http")
    print(f"✅ Streamlit app will be live at: {public_url}")

    # ----------------------------
    # Start the Streamlit app in the background
    # ----------------------------
    os.system(f"streamlit run {app_path} &>/content/logs.txt &")

    # ----------------------------
    # Wait a few seconds to ensure the app starts
    # ----------------------------
    time.sleep(3)

    return public_url

def shutdown_streamlit_and_ngrok():
    """
    Shut down any running Streamlit app and terminate all ngrok tunnels in a Colab session.

    This function performs:
    ------------------------
    - Kills all active ngrok tunnels using the pyngrok API
    - Terminates any background Streamlit processes using `pkill`
    - Provides feedback via console print statements
    """

    import os
    from pyngrok import ngrok

    print("⛔ Shutting down Streamlit and ngrok...")

    # ----------------------------
    # Kill all active ngrok tunnels
    # ----------------------------
    try:
        ngrok.kill()
        print("✅ ngrok tunnel(s) killed.")
    except Exception as e:
        print(f"⚠️ Error killing ngrok: {e}")

    # ----------------------------
    # Kill all background Streamlit processes
    # ----------------------------
    os.system("pkill -f streamlit")
    print("✅ Streamlit processes killed.")

## 3.2 Running our Streamlit Application

Once we run the `launch_streamlit_in_colab("data/streamlit-hello-world.py")` command in the following cell and you click on the shared URL link - we should see the following screens (if all things go well!)

> ✅ Make sure to click on the URL which looks like this: `"https://<some-random-string>.ngrok-free.app"` instead of `"http://localhost:8501"`

After clicking on this URL - you should see this welcome page from Ngrok with your own unique random URL which should match what the one you clicked on from the Colab notebook. It's fine to click on `Visit Site` to proceed!

![streamlit-hello-world-ngrok](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-ngrok.png)

You should first be welcomed by an authentication page.

![streamlit-hello-world-login](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-login.png)

Then following this - you should see text input asking for a prompt.

![streamlit-hello-world-prompt](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-prompt.png)

Finally - if all goes well, you should see something like this output - try it out using your own prompts!

![streamlit-hello-world-success](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/streamlit-hello-world-success.png)

The source code for the Hello World app can be viewed within the Streamlit app directly - and also in the GitHub repo for this course:

* https://github.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/blob/main/streamlit-hello-world.py

After you're done with this hello world example - you can run the cell directly below to stop running the Streamlit application and continue with our tutorial!

## 3.3 Launch Streamlit App

Run the following cell to get started!

> ✅ Make sure to click on the URL which looks like this: `"https://<some-random-string>.ngrok-free.app"` instead of `"http://localhost:8501"`

If anything goes wrong or you see an error about Ngrok only allowing one live tunnel at a time - simply run the `shutdown_streamlit_and_ngrok()` command in the following cell below and it everything should resolve itself!

In [ ]:
# Launch the Streamlit app in Colab
launch_streamlit_in_colab("data/streamlit-hello-world.py")

## 3.4 Shutdown Streamlit and Ngrok

Once you're ready to move onto the next component - we can shutdown our Streamlit app and free up the Ngrok tunnel.

In [ ]:
# Shutdown the Streamlit app in Colab
shutdown_streamlit_and_ngrok()

# 4. Pure Python LLM App

In the next cell - we'll run a Streamlit app which details a simple `Retrieval-Augmented-Generation` (RAG) pipeline using pure Python only.

![design-basic-app](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/design-basic-app.png)

This Streamlit app is a **Retrieval-Augmented Generation (RAG)** system that answers natural language travel queries about California.

It combines:
- Local semantic search using SentenceTransformer + FAISS  
- Cloud-based reasoning with OpenRouter's Mistral LLM  
- Structured data on locations and tour products

---

## How It Works

1. **User submits a travel question**  
   Example: “What are some scenic spots in Yosemite?”

2. **Local Semantic Search (FAISS + Embeddings)**  
   The app searches over a dataset of location descriptions using vector similarity.

3. **Tour Product Matching**  
   The app finds the most relevant tour products based on matched locations.

4. **LLM Response via OpenRouter API**  
   Retrieved context and products are sent to a hosted Mistral model to:
   - Generate a structured answer
   - Suggest 3 follow-up questions

5. **Interactive Chat Interface**  
   Users can click follow-up suggestions to continue the conversation.

---

## Key Components

| Component               | Purpose                                               |
|-------------------------|--------------------------------------------------------|
| `SentenceTransformer`   | Encodes queries and location/product text as embeddings |
| `FAISS`                 | Finds the most similar documents by cosine similarity   |
| `OpenRouter API`        | Sends prompts to a hosted Mistral LLM                  |
| `Streamlit`             | Manages UI and chat interactions                       |
| `products.csv`          | Tour product descriptions and metadata                 |
| `locations.csv`         | Descriptions of California travel destinations         |

---

## Retrieval-Augmented Generation (RAG)

This app demonstrates a classic RAG pipeline:

- **Retrieve**: Use FAISS to find top location matches  
- **Augment**: Insert retrieved context into an LLM prompt  
- **Generate**: Let the model answer the question and suggest follow-ups

---

## Login and Session Memory

- Password protection using `.env` configuration  
- `st.session_state` is used to remember:
  - Chat history
  - Suggested follow-ups
  - Retrieved context and products

---

## Example Output Format

```markdown
**Answer:** Yosemite is known for granite cliffs and waterfalls...

**Suggested Follow-Ups:**
- What are the best hikes for beginners?
- Can I do a day trip from San Francisco?
- What wildlife can I see in Yosemite?

In [ ]:
# Launch the Streamlit app in Colab
launch_streamlit_in_colab("data/streamlit-python-rag.py")

In [ ]:
# Shutdown the Streamlit app in Colab
shutdown_streamlit_and_ngrok()

# 5. LangChain App

In the next cell - we'll run a Streamlit app which extends our pure Python app by replacing certain components using `LangChain`


In [ ]:
# Run LangChain app
launch_streamlit_in_colab("data/streamlit-langchain-rag.py")


## App Comparisons

This outlines the key differences between the original Python-only RAG travel assistant and the updated LangChain-based version. It focuses on what learners should look out for when reading the source code.

---

| Feature               | Python RAG App                                  | LangChain Version                                      |
|------------------------|--------------------------------------------------|---------------------------------------------------------|
| Vector Search          | Manual FAISS index + SentenceTransformer        | LangChain `FAISS` vectorstore abstraction              |
| Embedding Model        | Loaded and applied manually                     | Handled via `HuggingFaceEmbeddings` object             |
| Prompt Construction    | f-strings and manual formatting                 | `ChatPromptTemplate` with structured system/human roles|
| LLM API Usage          | Direct OpenAI-compatible API via `openai`       | `ChatOpenAI` from LangChain                            |
| Memory / History       | Manual tracking in `st.session_state`           | `ConversationBufferMemory` + `StreamlitChatMessageHistory` |
| Chain Logic            | Custom function to call LLM                     | Encapsulated in `LLMChain`                             |
| Output Parsing         | Manual split on markdown delimiters             | Still manual (same logic reused)                       |

---

## Key Enhancements in the LangChain Version

### 1. Prompt Management
- Python app uses plain strings to define prompts.
- LangChain uses `ChatPromptTemplate`, separating roles (`system`, `human`) and keeping the prompt modular and readable.

### 2. LLM Calls
- Python app constructs API requests manually.
- LangChain wraps LLM usage into `LLMChain`, managing context, input variables, and memory under the hood.

### 3. Memory Integration
- Original app stores chat history manually using session state.
- LangChain provides a structured memory system that integrates with the chain directly, simplifying history reuse.

### 4. Vector Store
- In the original, embeddings are encoded and searched with raw FAISS APIs.
- In LangChain, documents are wrapped in `Document` objects and indexed with a single line:  
  ```python
  FAISS.from_documents(docs, embedder)
  ```

---

## Learner Takeaways

| Topic                      | Why It Matters                                                |
|----------------------------|---------------------------------------------------------------|
| Prompt Engineering         | LangChain improves readability and makes prompts reusable.   |
| Chat Memory                | LangChain handles turn history without extra logic.           |
| Abstractions vs. Control   | Python app gives full control; LangChain offers clean structure. |
| Chain Composition          | LangChain chains are modular and composable.                  |
| Debugging and Transparency | Both approaches expose full prompt and output when needed.    |

---

## Code Differences to Inspect

| Section                    | What to Compare                                               |
|----------------------------|---------------------------------------------------------------|
| `get_response_and_followups()` vs. `rag_chain.invoke()` | LLM invocation logic         |
| Prompt formatting          | f-strings vs. `ChatPromptTemplate`                            |
| Vector search logic        | Manual FAISS search vs. `similarity_search()`                |
| Memory structure           | `st.session_state["chat_history"]` vs. LangChain memory object |
| Follow-up parsing          | Identical logic reused in both implementations               |

In [ ]:
# Shutdown the Streamlit app in Colab
shutdown_streamlit_and_ngrok()

# 5. LangGraph Agentic AI

In the next cell - we'll run a Streamlit app which further extends upon our static AI workflows using `LangChain` to an autonomous agentic AI framework using `LangGraph`


In [ ]:
# Run LangGraph app
launch_streamlit_in_colab("data/streamlit-langgraph-rag.py")


Now we can compare the three implementations of the Explore California AI Travel Assistant:

- Python-only RAG app
- LangChain-powered RAG app
- LangGraph-powered agentic app

---

## Overview

| Feature               | Python RAG App         | LangChain App              | LangGraph App                      |
|------------------------|------------------------|-----------------------------|-------------------------------------|
| Vector Search          | Manual FAISS           | LangChain FAISS wrapper     | LangGraph FAISS wrapper             |
| Embeddings             | SentenceTransformer    | `HuggingFaceEmbeddings`     | `HuggingFaceEmbeddings`             |
| Prompt Handling        | Manual strings         | `ChatPromptTemplate`        | LLM prompt built inside tool node   |
| LLM Integration        | Raw OpenAI client       | `ChatOpenAI`                | `ChatOpenAI`                        |
| Memory/History         | Custom session state   | `ConversationBufferMemory`  | Passed explicitly in state graph    |
| Orchestration          | Manual logic           | `LLMChain`                  | `StateGraph` with conditional flow  |
| Follow-up Handling     | Manual UI logic        | Button + LLM                | Embedded in graph output state      |

---

## What Makes LangGraph Different?

- **Graph-Based Execution**: Logic is encoded as a conditional flow graph using `StateGraph`, enabling dynamic control over the sequence of tools.
- **Tool Functions**: Each tool (`@tool`) is a modular, typed function. Tools represent semantic retrieval, product matching, or LLM generation.
- **State Management**: Entire app logic runs via a dictionary `AppState`, making flow reproducible and observable.
- **Follow-Up Flow**: LangGraph can bypass location retrieval if the user is asking a follow-up—controlled by a dynamic node.

---

## Visual Flow Differences

```
Python RAG:            query -> embed -> FAISS -> LLM -> UI

LangChain:             query -> FAISS -> LLMChain(memory+prompt) -> UI

LangGraph:             query -> entry_selector
                           ↳ search_locations → match_products → generate_answer
                           ↳ generate_answer (if follow-up)
```

---

## Key Takeaways

| Concept                  | LangGraph Emphasis                          |
|--------------------------|---------------------------------------------|
| Modularity               | Breaks up functionality into reusable tools |
| Graph-Oriented Thinking  | Encourages declarative, inspectable flows   |
| Tool Inputs/Outputs      | Fully typed and validated                   |
| Control Flow             | Uses `entry_selector` to branch logic       |
| Production Readiness     | Easier to extend with observability/logging |

In [ ]:
# Shutdown the Streamlit app in Colab
shutdown_streamlit_and_ngrok()

# 6. Advanced AI Application

In the next cell - we'll run a Streamlit app which further extends our `LangGraph` app to incorporate our logistic regression ML model and complex LLM based routing and summarization with the ability to save chat history and get personalized recommendations.

We've also extended this application to generate token usage as part of the overall LLM metadata response outputs so we can track usage.

![design-advanced-app](https://raw.githubusercontent.com/LinkedInLearning/applied-AI-and-machine-learning-for-data-practitioners-5932259/main/images/design-advanced-app.png)


In [ ]:
# Run advanced Streamlit LangGraph app
launch_streamlit_in_colab("data/streamlit-advanced-ai-app.py")


## Basic and Advanced App Comparison

The following compares two LangGraph-based versions of the Explore California AI Travel Assistant:

- **LangGraph (Basic)**: Simple conditional RAG flow for answering questions
- **LangGraph (Advanced)**: Enhanced multi-modal pipeline using attributes, predictions, and topic routing

---

| Feature                          | LangGraph Basic                        | LangGraph Advanced                                |
|----------------------------------|----------------------------------------|---------------------------------------------------|
| Retrieval                        | Locations only                         | Locations + semantic user attributes              |
| ML Integration                   | None                                   | Uses ML model for product prediction              |
| Topic Routing                    | Boolean flag (follow-up or not)        | LLM-generated decision (search vs. attributes)    |
| Context Sources                  | Always from query                      | Can come from ML prediction + product metadata    |
| Chat History Management          | Basic list                             | Per-chat sessions with metadata + topic label     |
| UI Complexity                    | Basic chat + follow-up                 | Full saved chat manager with reloadable sessions  |
| LLM Call                         | `llm.invoke()`                         | `llm.generate()` (with token usage tracking)      |
| Attributes Vector Index          | Not included                           | Semantic search over defined user attribute space |
| Extensibility                    | Medium                                 | High – modular tools and dynamic routing          |

---

## Key Enhancements in the Advanced App

### 1. Attribute-Based Personalization
- Queries like “I like scenic drives and local food” are routed to a tool that finds similar predefined attributes (using FAISS).
- These attributes are passed to a classifier that predicts the most relevant travel product.

### 2. Intelligent Routing
- A single LLM call decides the user intent: attribute-based suggestion, factual question, or follow-up.
- Also generates a topic label for session organization.

### 3. Chat Session Management
- Each conversation is stored with its context, product info, and topic.
- Sidebar allows learners to reload past conversations or start new ones.

### 4. Token Tracking
- Uses `llm.generate()` to record token usage for analysis and cost-awareness.

---

## Workflow Comparison

```
Basic:
query → entry_selector → (search → match → generate) or (generate only)

Advanced:
query → entry_selector
  ├──→ find_attributes → predict_product → get_product_locations → generate
  ├──→ search_locations → match_products → generate
  └──→ generate (follow-up)
```

---

## Key Takeaways

| Concept                        | Advanced App Highlights                         |
|--------------------------------|--------------------------------------------------|
| Modularity                     | Clear separation of tools (search, match, predict, generate) |
| Agentic Workflow               | Tools form nodes in a decision graph            |
| Personalization via ML         | Product recommendation based on binary features |
| Graph Routing via LLM          | Conditional control flow without hardcoding     |
| Reusability                    | Components can be swapped, extended, or reused  |
| Observability                  | Result view shows full LLM inputs/outputs       |

In [ ]:
# Shutdown the Streamlit app in Colab
shutdown_streamlit_and_ngrok()

# Next Steps

Now that you've explored multiple versions of the Explore California AI Travel App, here are some practical next steps to help you deepen your understanding and prepare your app for production use!

### 1. Explore the Codebase
- Each Streamlit app (`RAG`, `LangChain`, `LangGraph`, and `Advanced LangGraph`) is fully open-source.
- Inspect how components are defined, including embedding models, FAISS indexes, and LangChain/LangGraph workflows.
- Pay attention to how state is managed across user sessions and how tools are composed in LangGraph.

### 2. Deploy to Streamlit Cloud
- You can deploy any of the apps directly to Streamlit Community Cloud.
- Make sure to include a `.streamlit/secrets.toml` file with your `OPEN_ROUTER_API_KEY` and any required credentials.
- Optionally connect it to a GitHub repository to enable continuous updates.

### 3. Add Python Unit Tests
- Create unit tests for key components such as:
  - Semantic search functions (e.g., `search_locations`)
  - Product matching logic
  - ML-based predictions
  - LLM formatting and parsing functions
- Use `pytest` or `unittest`, and mock external calls (e.g., LLMs or FAISS) for faster test runs.

### 4. Refactor into Modular Components
- Split large monolithic `app.py` files into:
  - `models/` → ML model loading and prediction
  - `tools/` → LangGraph tools and utility functions
  - `components/` → Streamlit UI components
  - `graphs/` → LangGraph workflows
  - `services/` → LLM API interaction and embedding logic
- This makes the codebase easier to test, maintain, and extend.

### 5. Extend the Application
- Add multi-turn memory using LangChain agents or LangGraph memory nodes.
- Add feedback logging or analytics to capture query usage and LLM performance.
- Add multilingual support with translation layers or region-specific filters.
- Extend ML product recommendations to use user history, clustering, or collaborative filtering.

---

By breaking the app into composable, testable, and deployable pieces, you're well on your way to building production-grade AI applications using RAG, LangChain, and LangGraph.


# Final Steps

Run the following cell block to remove all your keys and sensitive information from the Google Colab instance!

In [ ]:
# Cleanup - kill any ngrok tunnels and delete sensitive .env file
import os
import dotenv
from pyngrok import ngrok

if os.path.exists(".env"):
    
    # Load environment variables from .env file
    dotenv.load_dotenv()

    # Check if NGROK_AUTH_TOKEN is set and stop any existing tunnels
    if os.getenv("NGROK_AUTH_TOKEN"):
        ngrok.set_auth_token(os.getenv("NGROK_AUTH_TOKEN"))

        # Stop any existing Ngrok tunnels
        ngrok.kill()
        print("✅ Ngrok tunnels stopped.")
    
    # Remove the .env file
    os.remove(".env")
    print("✅ .env file removed safely.")